In [ ]:
from llama_index.core import Settings 
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner
import os 

In [23]:
llm = Groq(
    model="llama-3.3-70b-versatile",
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [25]:
def calcular_imposto_renda(rendimento: float) -> str:
    if rendimento <= 22847.76:
        return "Isento de Imposto de Renda"
    elif rendimento <= 33919.80:
        imposto = (rendimento - 22847.76) * 0.075
    elif rendimento <= 45012.60:
        imposto = (rendimento - 33919.80) * 0.15 + 826.29
    elif rendimento <= 55976.16:
        imposto = (rendimento - 45012.60) * 0.225 + 1428.14
    else:
        imposto = (rendimento - 55976.16) * 0.275 + 2203.91

    return f"Imposto de Renda devido: R$ {imposto:.2f}"

In [26]:
ferramenta_calculo_imposto = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="calcular_imposto_renda",
    description=(
        "Calcula o Imposto de Renda devido com base no rendimento anual. "
        "Argumento: rendimento (float) - Rendimento anual em reais. "
        "Retorna uma mensagem indicando se é isento ou o valor do imposto a pagar."
    ),

)

In [ ]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    llm=llm,
    tools=[ferramenta_calculo_imposto],
    verbose=True,
    allow_parallel_tool_calls=True, # Permite chamadas paralelas de ferramentas
)

In [ ]:
agent_imposto = AgentRunner(agent_worker_imposto)


In [ ]:
response = agent_imposto.chat(
    "Qual é o imposto de renda devido para um rendimento anual de R$ 50000.00?",
)

In [ ]:
import arxiv
def consulta_artigos(title: str) -> str:
    """
    Consulta artigos na base dados ArXiv e retorna resultados formatados.
    """

    # O arqiv é uma base de dados de artigos científicos
    buscar = arxiv.Search(
        query=title,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance,
    )

    reulstados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in buscar.results()
    ]

    return "\n\n".join(reulstados) if reulstados else "Nenhum artigo encontrado."

In [52]:
consulta_artigos_tool = FunctionTool.from_defaults(
    fn=consulta_artigos,
    name="consulta_artigos",
    description=(
        "Consulta artigos na base de dados ArXiv com base no título fornecido. "
        "Argumento: title (str) - Título do artigo a ser pesquisado. "
        "Retorna uma lista formatada de artigos encontrados."
    ),
)

In [ ]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    llm=llm,
    tools=[ferramenta_calculo_imposto, consulta_artigos_tool],
    verbose=True,
    # Desabilita chamadas paralelas de ferramentas, pois eu quero sequencialmente
    allow_parallel_tool_calls=False,  
)

In [56]:
agent = AgentRunner(agent_worker)
response2 = agent.chat(
    "Quais artigos existem sobre 'Machine Learning'?",
)

Added user message to memory: Quais artigos existem sobre 'Machine Learning'?
=== Calling Function ===
Calling function: consulta_artigos with args: {"title": "Machine Learning"}
=== Function Output ===
Encountered error: name 'arxiv' is not defined
=== Calling Function ===
Calling function: consulta_artigos with args: {"title": "Machine Learning"}
=== Function Output ===
Encountered error: name 'arxiv' is not defined
=== Calling Function ===
Calling function: consulta_artigos with args: {"title": "Machine Learning"}
=== Function Output ===
Encountered error: name 'arxiv' is not defined
=== Calling Function ===
Calling function: consulta_artigos with args: {"title": "Machine Learning"}
=== Function Output ===
Encountered error: name 'arxiv' is not defined
=== Calling Function ===
Calling function: consulta_artigos with args: {"title": "Machine Learning"}
=== Function Output ===
Encountered error: name 'arxiv' is not defined
